In [2]:
!pip install neat-python

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [6]:
import neat
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Загрузка данных
dataset_1 = pd.read_csv('/Users/macbook/Desktop/ВУЗ/Практикум по программированию/lab_2-5/lab_2/dataset_1.csv')  # Маленький датасет
dataset_2 = pd.read_csv('/Users/macbook/Desktop/ВУЗ/Практикум по программированию/lab_2-5/lab_2/dataset_12.csv')  # Большой датасет

def one_hot_encode_pandas(df):
    # Выбираем только категориальные колонки (не числовые)
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    # Применяем One-Hot Encoding
    df_encoded = pd.get_dummies(df, columns=categorical_cols)
    return df_encoded

# Применяем к обоим датасетам
dataset_1_encoded = one_hot_encode_pandas(dataset_1)
dataset_2_encoded = one_hot_encode_pandas(dataset_2)

# Предобработка данных
def prepare_data(data):
    X = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return train_test_split(X, y, test_size=0.2, random_state=42)

X1_train, X1_test, y1_train, y1_test = prepare_data(dataset_1_encoded)
X2_train, X2_test, y2_train, y2_test = prepare_data(dataset_2_encoded)

In [23]:
dataset_1_encoded

,Obj1_Feature1_binary,Obj1_Feature4_numeric,Obj1_Feature5_binary,Obj2_Feature1_binary,Obj2_Feature4_numeric,Obj2_Feature5_binary,Collision,Obj1_Feature2_nominal_A,Obj1_Feature2_nominal_B,Obj1_Feature2_nominal_C,...,Obj1_Feature3_ordinal_Medium-High,Obj1_Feature3_ordinal_Medium-Low,Obj2_Feature2_nominal_A,Obj2_Feature2_nominal_B,Obj2_Feature2_nominal_C,Obj2_Feature3_ordinal_High,Obj2_Feature3_ordinal_Low,Obj2_Feature3_ordinal_Medium,Obj2_Feature3_ordinal_Medium-High,Obj2_Feature3_ordinal_Medium-Low
0,1,-1.193960,1,0,-0.171083,1,0,True,False,False,...,False,False,False,True,False,False,False,False,True,False
1,0,-0.586181,0,1,-0.231838,0,0,False,True,False,...,False,False,True,False,False,True,False,False,False,False
2,1,-0.799870,0,1,-0.052146,1,0,False,False,True,...,False,False,True,False,False,False,False,False,False,True
3,0,0.165268,0,1,0.135464,0,1,True,False,False,...,True,False,False,True,False,False,True,False,False,False
4,1,-0.718194,1,0,0.620215,0,0,True,False,False,...,False,False,False,False,True,True,False,False,False,False
5,0,-0.548686,1,0,1.494772,0,1,True,False,False,...,True,False,False,True,False,True,False,False,False,False
6,1,1.038304,0,0,-0.135654,1,1,True,False,False,...,True,False,False,False,True,False,False,False,False,True
7,1,-1.190019,1,0,-0.757278,0,0,False,False,True,...,False,True,False,False,True,False,False,True,False,False
8,1,-0.035020,1,1,2.145907,1,1,True,False,False,...,False,False,False,True,False,False,False,True,False,False
9,0,0.246568,1,1,0.568108,0,1,True,False,False,...,False,True,True,False,False,False,False,True,False,False


In [24]:
dataset_2_encoded

,Obj1_Feature1_binary,Obj1_Feature4_numeric,Obj1_Feature5_binary,Obj1_Feature8_numeric,Obj1_Feature9_binary,Obj1_Feature12_numeric,Obj2_Feature1_binary,Obj2_Feature4_numeric,Obj2_Feature5_binary,Obj2_Feature8_numeric,...,Obj2_Feature7_ordinal_Medium-High,Obj2_Feature7_ordinal_Medium-Low,Obj2_Feature10_nominal_A,Obj2_Feature10_nominal_B,Obj2_Feature10_nominal_C,Obj2_Feature11_ordinal_High,Obj2_Feature11_ordinal_Low,Obj2_Feature11_ordinal_Medium,Obj2_Feature11_ordinal_Medium-High,Obj2_Feature11_ordinal_Medium-Low
0,1,0.044279,0,0.559199,1,-0.427377,0,-0.893318,0,-0.703389,...,False,False,False,False,True,False,False,True,False,False
1,1,0.881690,1,-1.203021,1,-1.052097,1,0.505079,0,-0.929075,...,True,False,False,True,False,False,False,False,True,False
2,1,-0.631630,1,-0.549399,0,0.880429,1,-0.085038,0,-1.541565,...,False,False,False,False,True,False,False,True,False,False
3,1,-0.845899,1,0.309736,1,-0.748742,0,-0.423804,1,-0.480035,...,False,False,False,True,False,False,True,False,False,False
4,1,1.726137,1,0.062454,0,-0.532958,1,0.041119,0,0.826193,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,1,0.140016,0,-0.038066,1,-0.702560,1,-0.950488,0,-0.018678,...,False,False,False,True,False,False,False,True,False,False
2996,0,-1.320258,1,-0.104930,0,-1.957735,1,0.614075,1,1.372446,...,True,False,False,False,True,False,False,False,False,True
2997,1,-1.274997,1,-0.995226,1,1.134566,0,-0.487352,0,0.924074,...,False,True,False,False,True,True,False,False,False,False
2998,1,-0.554627,0,-1.223008,0,-0.837496,1,-0.763711,1,-0.281743,...,False,False,False,True,False,False,False,False,False,True


In [28]:
# Конфигурация NEAT для маленького датасета
config_1 = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config_1.txt')

def eval_genomes_1(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        outputs = []
        for xi in X1_train:
            output = net.activate(xi)
            outputs.append(np.argmax(output))
        accuracy = accuracy_score(y1_train, outputs)
        genome.fitness = accuracy

# Создание популяции
population_1 = neat.Population(config_1)
population_1.add_reporter(neat.StdOutReporter(True))
stats_1 = neat.StatisticsReporter()
population_1.add_reporter(stats_1)

# Запуск обучения
winner_1 = population_1.run(eval_genomes_1, 50)  # 50 поколений


 ****** Running generation 0 ****** 

Population's average fitness: 0.03200 stdev: 0.04048
Best fitness: 0.17500 - size: (60, 660) - species 1 - id 13
Average adjusted fitness: 0.032
Mean genetic distance 2.005, standard deviation 0.132
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100      0.2    0.032     0
Total extinctions: 0
Generation time: 0.959 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.03175 stdev: 0.05048
Best fitness: 0.30000 - size: (60, 659) - species 1 - id 182
Average adjusted fitness: 0.032
Mean genetic distance 2.225, standard deviation 0.113
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100      0.3    0.032     0
Total extinctions: 0
Generation time: 0.970 sec (0.965 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.03875 st

CompleteExtinctionException: 